### Wetbulb Temperature
From this paper:
https://journals.ametsoc.org/view/journals/apme/50/11/jamc-d-11-0143.1.xml

In [1]:
def Wetbulb(tmp, rh):
    wetbulb_tmp = ( tmp * np.arctan(0.151977 * (rh + 8.313659)**(1/2)) ) + np.arctan(tmp + rh) - np.arctan(rh - 1.676331) + (0.00391838 * (rh)**(3/2)) * np.arctan(0.023101 * rh) - 4.686035
    return wetbulb_tmp

In [3]:
# Validation. It should be 13.7
import numpy as np

tmp = 20
rh = 50

print(Wetbulb(tmp,rh))

13.699341968988136


### Heat Index
From Maile's climate-map repository:
* https://github.com/mailesasaki/climate_map/blob/main/calculations/calculations.py

In [ ]:
def Heat_Index(T, RH):
    """
    https://www.wpc.ncep.noaa.gov/html/heatindex_equation.shtml

    Calculates heat index for an array
    
    Inputs:
        RH (DataArray) - Should be in decimal format
        T  (DataArray) - Should be in Kelvins
        
    Outputs:
        hi_alone (DataArray) - Heat index array (in F)
    """
    # Convert to Fahrenheit
    T_F = ((T - 273.15) * 1.8) + 32

    # Convert to relative humidity
    RH_p = RH * 100
    RH_p = RH_p.rename('relative_humidity')
    
    # Standard heat index
    heat_index = 0.5 * (T_F + 61.0 + ((T_F-68.0)*1.2) + (RH_p*0.094))
    heat_index = heat_index.rename('heat_index')

    # Combining temperature, relative humidity, and heat index into a dataset
    hi_set = xr.combine_by_coords((heat_index,T_F,RH_p))
        
    # Heat index for heat index above 80
    heat_index_80 = (-42.379 + 2.04901523*T_F + 10.14333127*RH_p - 0.22475541*T_F*RH_p 
          - 6.83783e-3*T_F**2 - 5.481717e-2*RH_p**2 + 1.22874e-3*T_F**2*RH_p 
          + 8.5282e-4*T_F*RH_p**2 - 1.99e-6*T_F**2*RH_p**2)
    hi_set['heat_index>80'] = heat_index_80
    
    # Replacing heat indices above 80 with the new equation
    hi_set['heat_index'] = xr.where(hi_set['heat_index']>80,
                                    hi_set['heat_index>80'],
                                    hi_set['heat_index']
                                    )
    
    # Heat index for relative humidity under 13% and temps between 80 and 112 F
    heat_index_13 = heat_index_80 - ((13-RH_p)/4) * np.sqrt((17 - abs(T_F - 95))/17)
    hi_set['heat_index_RH<13'] = heat_index_13
    
    hi_set['heat_index'] = xr.where(((hi_set['relative_humidity']<13) & 
                                         (hi_set['t2m']>80) & 
                                         (hi_set['t2m']<112)),
                                    hi_set['heat_index_RH<13'],
                                    hi_set['heat_index'])
    
    # Heat index for relative humidity over 85% and temps between 80 and 87 F
    heat_index_85 = heat_index_80 + ((RH_p-85)/10) * ((87-T_F)/5)
    hi_set['heat_index_RH>85'] = heat_index_85
    hi_set['heat_index'] = xr.where(((hi_set['relative_humidity']>85) & 
                                         (hi_set['t2m']>80) & 
                                         (hi_set['t2m']<87)),
                                    hi_set['heat_index_RH>85'],
                                    hi_set['heat_index'])
    
    # Picking out the heat index dataarray alone
    hi_alone = hi_set['heat_index']

    return hi_alone